# Results of ResNet18 Trained on CIFAR100

In this notebook I display results in terms of accuracy on test data for torchvisions' ResNet18 models trained on the CIFAR100 dataset using various regularization techniques. The CIFAR100 data is preprocessed by normalizing using mean [0.5071, 0.4865, 0.4409] and variance [0.2009, 0.1984, 0.2023]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization (there is no dropout for the standard ResNet models). The weights area initialized by the pytorch standard. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 50 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR100
from model_classes import ResNet18
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR100()

# Summary of model
summary_model = ResNet18().to(device)
summary(summary_model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified


c:\ProgramData\Anaconda3\envs\bioai\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\bioai\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [3]:
# Load models
dataset = "cifar100"

model_names_set = [
    "model_no_reg",
    "model_l2",
    "model_jacobi",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,45.19 +/- 0.785
1,model_l2,49.802 +/- 0.416
2,model_jacobi,44.888 +/- 0.733
3,model_jacobi_no_dropout,45.332 +/- 1.424
4,model_svb,50.208 +/- 0.278


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [4]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 114.93it/s]


In [5]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                       0.010                \
                        mean conf_interval          mean conf_interval   
model_no_reg_0  34537.951372   6257.114262  63299.008045   7247.629571   
model_l2_0      36219.951587   7649.736672  63696.801534   7437.800825   
model_jacobi_0  31266.460329   6555.674364  58517.742071   7423.093863   
model_svb_0      34968.54219   6702.421845  59955.664632   6473.634493   
model_no_reg_1  28356.032061   6332.657798  57244.873369   7727.023416   
model_l2_1      35691.443763   7217.853081  62835.843848   7245.351875   
model_jacobi_1  31520.592885   6841.214468  51395.050592    7606.57434   
model_svb_1      40666.47299   7182.636728  64104.706067   6782.036178   
model_no_reg_2  24390.982257   6989.551275  56130.565187   8309.447133   
model_l2_2      32559.341091   6337.222431  61304.567541   7485.798907   
model_jacobi_2  31957.874839   7174.533769  57194.066772   8066.095815   
model_svb_2     35658.758732   5959.040003  68028.372303   6748.635627   
model_no_reg_3  34458.820336   5817.244708  69497.876605   7846.432192   
model_l2_3       35631.24736   7031.783092  59626.015905   6534.028121   
model_jacobi_3  37062.060587   7932.446652  62484.783824   9256.328353   
model_svb_3     35313.661896   6456.772961  59175.429426   7766.905055   
model_no_reg_4  32430.986567   7010.250447  57344.723564   7588.208304   
model_l2_4      35265.222274   6984.178489  62357.031307   7010.817749   
model_jacobi_4  28531.400491   6229.337437  59137.375411   7228.946005   
model_svb_4     33383.637791   7443.123603  61599.474253   8678.034381   

                       0.001                
                        mean conf_interval  
model_no_reg_0  85673.417256   4819.611752  
model_l2_0      29928.471074   3470.837805  
model_jacobi_0  25483.988274   2413.105851  
model_svb_0     26305.141842   4562.371967  
model_no_reg_1  11229.180458   1335.748036  
model_l2_1      49750.147578   3405.917244  
model_jacobi_1  57357.966397   2202.248122  
model_svb_1     49110.399234   4805.353141  
model_no_reg_2  57768.057585   3728.250722  
model_l2_2      66690.972456   4492.696284  
model_jacobi_2  56499.980251   3101.187022  
model_svb_2     41852.785603   3263.276839  
model_no_reg_3  42786.706967   2831.654569  
model_l2_3      45084.998918    4734.80794  
model_jacobi_3  59932.402553   4077.306158  
model_svb_3     35464.739343   2702.335463  
model_no_reg_4  65965.653578   4704.516423  
model_l2_4      67736.534573   4207.146874  
model_jacobi_4  41635.870722   3135.782155  
model_svb_4     14411.076724   2239.135959

#### Anisotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# # Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_jacobi_no_dropout_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 49.63it/s]


In [7]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                   0.010                   0.001  \
                    mean conf_interval      mean conf_interval      mean   
model_no_reg_0  39959.34   7068.210939  62910.98   7542.356867  91918.04   
model_l2_0      46983.96   7283.086637   77676.7   7275.786872  31806.14   
model_jacobi_0  45091.76   8029.893827  72474.28   8786.634935   28393.0   
model_svb_0     31888.22   6731.953659  69696.82   7846.423376  30645.78   
model_no_reg_1  37063.98   6932.241749   66888.9   6817.167842   12154.6   
model_l2_1      40313.46   6963.909359  71723.12   6959.735172  56278.18   
model_jacobi_1  34041.28   7775.304472  54999.74   7693.097805  63075.98   
model_svb_1     36524.82   7605.292422   68854.4   8922.262358  54297.88   
model_no_reg_2  30483.74   7266.933399  67582.04    7546.07836  67282.48   
model_l2_2      41140.36   7869.728082  69744.54   7608.703044   71978.2   
model_jacobi_2  40326.38   7497.711289  74270.86   8446.849419  61036.56   
model_svb_2     45009.14   8560.267628  81895.86   8730.092008   52227.7   
model_no_reg_3  32513.64   8040.870261  72052.76   9394.014843   47868.2   
model_l2_3      33589.26   8105.275156  64592.12   8063.841101  56932.08   
model_jacobi_3   38729.6   8040.522181  70835.12   8473.494394   69317.6   
model_svb_3     36182.66     8053.1249  69587.12   7635.675566  40587.38   
model_no_reg_4  39128.88   8318.049874  67390.34   8345.476755  79701.08   
model_l2_4      40052.76   6950.269017   71202.2   8150.320322  72913.52   
model_jacobi_4  33531.22   7942.393836  67242.26   9168.409116  47146.68   
model_svb_4     41555.48   7165.367248  69276.12   7521.905955  15132.22   

                              
               conf_interval  
model_no_reg_0   4948.076173  
model_l2_0       3984.679774  
model_jacobi_0   2802.096564  
model_svb_0      5685.723924  
model_no_reg_1   1399.479788  
model_l2_1       3702.176046  
model_jacobi_1   2283.630591  
model_svb_1      5934.049782  
model_no_reg_2   3870.306648  
model_l2_2       4407.357241  
model_jacobi_2   3594.562589  
model_svb_2      4576.128298  
model_no_reg_3   2873.338712  
model_l2_3       5419.864512  
model_jacobi_3   4940.940686  
model_svb_3      3326.719197  
model_no_reg_4    3867.51535  
model_l2_4       4688.349012  
model_jacobi_4   3499.387718  
model_svb_4      2538.314016